Combinaison de classifieurs 
===========================

Il est toujours utile de combiner de multiples classifieurs pour améliorer les performances des systèmes de classification.


Types de combinaisons
--------------------

Il existe plusieurs façons de combiner des classifieurs.
- On peut faire voter chaque classifieur. Et classer un exemple par vote majoritaire, dans la classe qui a recueilli le maximum de votes. On peut pondérer ces votes en fonction de la confiance que l'on a dans la performance du classifieur. En général on préfère utiliser au moins 3 classifieurs.
- On peut combiner les scores des différents classifieurs de façon plus complexe. On calcule un score global par classe à partir des scores de cette classe obtenus par chacun des classiieurs. On en calcule la moyenne arithmétique ou géométrique, le maximum, la variance. Et on classe en fonction de ce score global.



Quels modèles combiner ?
----------------------

Il est d'usage de considérer que plus les classifieurs que l'on combine sont différents meilleure est leur combinaison. Il existe plusieurs façons d'obtenir des classifieurs divers.
- On peut faire varier les données d'apprentissage. SI on dispose de N exemples d'apprentissage on crée K ensembles d'apprentissage de taille N en tirant avec remise dans l'ensemble original de train un ensemble d'apprentissage de N exemples. Et on apprend un modèle à partir de chaque ensemble d'apprentissage.
- On utilise des modèles dont on fait varier certains paramètres. Par exemple on peut apprendre des Réseaux de Neurones avec différents nombred de couches cachées, des filtres de convolution différents, des réseaux avec et sans convolutions etc. On peut aussi utiliser la mpeme architecture et compter sur l'initialisation aléatoire et la convergence vers un minimum local pour assurer une certaine diversité.
- On utilise différents types de classifieurs, svm, RN, etc

Votre travail
------------
Le code qui vous est donné ci-dessous permet de récupérer un réseau de neurones appris et de l'utiliser en TEST (sans réapprentissage) pour obtenir ses prédictions sur les données de test. Vous devrez utiliser ce code pour concevoir un système de classification plus performant sur les données d'émotions.
Vous avez donc à :
- Définir les modèles que vous souhaitez apprendre sur les données de train. EN cherchant à privilégier une certaine diversité dans les modèles. Vous devez enseuite apprendre ces modèles avec caffe.
- Utiliser le code ci-dessous pour récupérer les prédictions (ou en modifiant un peu le code, les scores par classe et par exemple) de vos modèles.
- Définir la façon de combiner les scores et écrire le code python correspondant. Puis évaluer en test la performance obtenue.



Récupération des prédictions d'un réseau Caffe 
====================================================================


In [13]:
import numpy as np

print np.__version__
import matplotlib.pyplot as plt
%matplotlib inline

import sys
caffe_root="/home/christian/caffe/build/"
sys.path.insert(0, caffe_root + 'python')
TP_root="/home/christian/deep/rendu/"

print sys.path 
print caffe_root

import caffe
import pickle
from sklearn import svm

nb_donnees=3500 # Nb données de test
outfile_labels = TP_root+"labels"
outfile_public_labels = TP_root+"public_labels"

#caffe.set_mode_cpu()

1.8.2
['/home/christian/caffe/build/python', '/home/christian/caffe/build/python', '', '/home/christian/caffe/python', '/home/christian/caffe/build/python', '/home/christian/deep/rendu', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/christian/.local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/PILcompat', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/lib/pymodules/python2.7', '/usr/lib/python2.7/dist-packages/ubuntu-sso-client', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/home/christian/.ipython']
/home/christian/caffe/build/


In [14]:
##RUN LENET et génère ses prédictions sur le private test
MODEL_FILE_LENET = TP_root+"model/lenet_train_test.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_train_test_iter_10000.caffemodel"
outfile_lenet = TP_root+"predictions/predictions_lenet"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_lenet+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_lenet+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_lenet,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.520571428571


In [15]:
##RUN LENET_monoconv et génère ses prédictions sur le private test
MODEL_FILE_LENET = TP_root+"model/lenet_monoconv.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_monoconv_iter_10000.caffemodel"
outfile_monoconv = TP_root+"predictions/predictions_monoconv"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_monoconv+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_monoconv+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_monoconv,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.479714285714


In [16]:
##RUN LENET_tripleconv et génère ses prédictions sur le private test
MODEL_FILE_LENET = TP_root+"model/lenet_tripleconv.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_tripleconv_iter_10000.caffemodel"
outfile_tripleconv = TP_root+"predictions/predictions_tripleconv"
derniere_couche="ip1"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_tripleconv+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_tripleconv+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_tripleconv,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.470285714286


In [17]:
##RUN LENET_3conv2ip et génère ses prédictions sur le private test
MODEL_FILE_LENET = TP_root+"model/lenet_3conv2ip.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_3conv2ip_iter_10000.caffemodel"
outfile_3conv2ip = TP_root+"predictions/predictions_3conv2ip"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_3conv2ip+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_3conv2ip+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_3conv2ip,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.454857142857


In [18]:
##RUN LENET_microsoft et génère ses prédictions sur le private test
## based on  http://research.microsoft.com/pubs/258194/icmi2015_ChaZhang.pdf
MODEL_FILE_LENET = TP_root+"model/lenet_microsoft.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_microsoft_iter_4000.caffemodel"
outfile_microsoft = TP_root+"predictions/predictions_microsoft"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoft+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoft+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_microsoft,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.468


In [19]:
##RUN LENET_microsoft et génère ses prédictions sur le private test
MODEL_FILE_LENET = TP_root+"model/lenet_microsoft.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_microsoftlr2_iter_10000.caffemodel"
outfile_microsoftlr2 = TP_root+"predictions/predictions_microsoftlr2"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoftlr2+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoftlr2+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_microsoftlr2,"wb"))
pickle.dump(Labels,open(outfile_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.459714285714


In [20]:
##RUN LENET et génère ses prédictions sur le public test
MODEL_FILE_LENET = TP_root+"model/public_lenet_train_test.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_train_test_iter_10000.caffemodel"
outfile_lenet = TP_root+"predictions/public_predictions_lenet"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_lenet+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_lenet+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_lenet,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.173142857143


In [21]:
##RUN LENET_monoconv et génère ses prédictions sur le public test
MODEL_FILE_LENET = TP_root+"model/public_lenet_monoconv.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_monoconv_iter_10000.caffemodel"
outfile_monoconv = TP_root+"predictions/public_predictions_monoconv"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_monoconv+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_monoconv+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_monoconv,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.178857142857


In [22]:
##RUN LENET_tripleconv et génère ses prédictions sur le public test
MODEL_FILE_LENET = TP_root+"model/public_lenet_tripleconv.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_tripleconv_iter_10000.caffemodel"
outfile_tripleconv = TP_root+"predictions/public_predictions_tripleconv"
derniere_couche="ip1"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_tripleconv+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_tripleconv+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_tripleconv,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.180285714286


In [23]:
##RUN LENET_3conv2ip et génère ses prédictions sur le public test
MODEL_FILE_LENET = TP_root+"model/public_lenet_3conv2ip.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_3conv2ip_iter_10000.caffemodel"
outfile_3conv2ip = TP_root+"predictions/public_predictions_3conv2ip"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_3conv2ip+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_3conv2ip+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_3conv2ip,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.182285714286


In [24]:
##RUN LENET_microsoft et génère ses prédictions sur le private test
## based on  http://research.microsoft.com/pubs/258194/icmi2015_ChaZhang.pdf
MODEL_FILE_LENET = TP_root+"model/public_lenet_microsoft.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_microsoft_iter_4000.caffemodel"
outfile_microsoft = TP_root+"predictions/public_predictions_microsoft"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoft+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoft+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_microsoft,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.178571428571


In [25]:
##RUN LENET_microsoftlr2 et génère ses prédictions sur le public test
MODEL_FILE_LENET = TP_root+"model/public_lenet_microsoft.prototxt"
PRETRAINED_LENET = TP_root+"pretrained/lenet_microsoftlr2_iter_10000.caffemodel"
outfile_microsoftlr2 = TP_root+"predictions/public_predictions_microsoftlr2"
derniere_couche="ip2"

net = caffe.Net(MODEL_FILE_LENET,PRETRAINED_LENET, caffe.TEST)

#Premier forward pour initier
out = net.forward()
pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoftlr2+"_batch_0","wb"))
taille_batch = net.blobs['data'].data.shape[0]
N_Batch = nb_donnees / taille_batch
Predictions = np.argmax(net.blobs[derniere_couche].data,1).reshape((1,taille_batch))
Labels = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))

#Suite des forward
for i in np.arange(N_Batch-1):
    out = net.forward()
    pickle.dump(net.blobs[derniere_couche].data, open(outfile_microsoftlr2+"_batch_"+str(i+1),"wb"))
    Predictions_tmp = np.argmax(net.blobs[derniere_couche].data,1).reshape((1, taille_batch))
    Labels_tmp = np.copy(net.blobs['label'].data[:]).reshape((1, taille_batch))
    Predictions = np.hstack((Predictions, Predictions_tmp))
    Labels = np.hstack((Labels, Labels_tmp))

#Sauvegarde les predictions et les labels
pickle.dump(Predictions, open(outfile_microsoftlr2,"wb"))
pickle.dump(Labels,open(outfile_public_labels,"wb"))

#Calcul de la précision
indices = (Labels==Predictions)
H= Labels[Labels==Predictions]
tr = (H.size +0.0) / (Labels.size +0.0)
print tr

#nettoie les variables
del net
del Predictions

0.164571428571


In [26]:
##PREDICTIONS où le vote de chaque NN est pondéré par sa précision sur le test
##Résultat sur le Private test
N_Batch = 35
taille_batch= 100
outfile_lenet=TP_root+"predictions/predictions_lenet"
outfile_monoconv=TP_root+"predictions/predictions_monoconv"
outfile_tripleconv=TP_root+"predictions/predictions_tripleconv"
outfile_3conv2ip=TP_root+"predictions/predictions_3conv2ip"
outfile_microsoft=TP_root+"predictions/predictions_microsoft"
outfile_microsoftlr2=TP_root+"predictions/predictions_microsoftlr2"

outfile_labels=TP_root+"labels"
Labels=pickle.load(open(outfile_labels,"r"))


Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_0","r"))
Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_0","r"))
Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_0","r"))
Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_0","r"))
Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_0","r"))
Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_0","r"))
Predictions_global = Predictions_lenet*0.5207 
Predictions_global += Predictions_monoconv*0.4797 
Predictions_global += Predictions_tripleconv*0.4702
Predictions_global += Predictions_3conv2ip * 0.4549 
Predictions_global += Predictions_microsoft * 0.468
Predictions_global += Predictions_microsoftlr2 * 0.4597

Predictions= np.argmax(Predictions_global,1).reshape((1, taille_batch))

for i in np.arange(N_Batch-1):
    Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_"+str(i+1),"r"))
    Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_"+str(i+1),"r"))
    Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_"+str(i+1),"r"))
    Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_"+str(i+1),"r"))
    Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_"+str(i+1),"r"))
    Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_"+str(i+1),"r"))
    Predictions_global = Predictions_lenet*0.5207 
    Predictions_global += Predictions_monoconv*0.4797
    Predictions_global += Predictions_tripleconv*0.4702 
    Predictions_global += Predictions_3conv2ip * 0.4549 
    Predictions_global += Predictions_microsoft * 0.468
    Predictions_global += Predictions_microsoftlr2 * 0.4597

    Predictions_tmp = np.argmax(Predictions_global,1).reshape((1, taille_batch))

    Predictions = np.hstack((Predictions, Predictions_tmp))       

print Predictions[0].shape
print Labels[0].shape
inc=0
for i in range(Labels.size):
    if (int(Labels[0][i]) == Predictions[0][i]):
        inc = inc +1

tr = (inc +0.0) / (Labels.size +0.0)
print tr
del Predictions

(3500,)
(3500,)
0.58


In [35]:
##train un SVM sur les predictions précédentes sur le private test
N_Batch = 35
taille_batch= 100
outfile_lenet=TP_root+"predictions/predictions_lenet"
outfile_monoconv=TP_root+"predictions/predictions_monoconv"
outfile_tripleconv=TP_root+"predictions/predictions_tripleconv"
outfile_3conv2ip=TP_root+"predictions/predictions_3conv2ip"
outfile_microsoft=TP_root+"predictions/predictions_microsoft"
outfile_microsoftlr2=TP_root+"predictions/predictions_microsoftlr2"

outfile_labels=TP_root+"labels"
Labels=pickle.load(open(outfile_labels,"r"))
Labels_vertical=np.zeros((3500,1))
for i in np.arange(3500):
    Labels_vertical[i,0]=Labels[0,i]
print Labels_vertical

Predictions_table=np.zeros((3500,6*7-1))

##Créer un tableau de prédictions [3500 * 41]. Ligne = entrées. Colonne = features (prédictions des NN mis bout à bout).
#Premier run initial
Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_0","r"))
Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_0","r"))
Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_0","r"))
Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_0","r"))
Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_0","r"))
Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_0","r"))
for i in np.arange(100):
    Predictions_table[i,0:6]=Predictions_lenet[i,0:6]
    Predictions_table[i,7:13]=Predictions_monoconv[i,0:6]
    Predictions_table[i,14:20]=Predictions_tripleconv[i,0:6]
    Predictions_table[i,21:27]=Predictions_3conv2ip[i,0:6]
    Predictions_table[i,28:34]=Predictions_microsoft[i,0:6]
    Predictions_table[i,35:41]=Predictions_microsoftlr2[i,0:6]

for i in np.arange(N_Batch-1):
    Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_"+str(i+1),"r"))
    Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_"+str(i+1),"r"))
    Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_"+str(i+1),"r"))
    Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_"+str(i+1),"r"))
    Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_"+str(i+1),"r"))
    Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_"+str(i+1),"r"))
    for j in np.arange(100):
        Predictions_table[(i+1)*100+j,0:6]=Predictions_lenet[j,0:6]
        Predictions_table[(i+1)*100+j,7:13]=Predictions_monoconv[j,0:6]
        Predictions_table[(i+1)*100+j,14:20]=Predictions_tripleconv[j,0:6]
        Predictions_table[(i+1)*100+j,21:27]=Predictions_3conv2ip[j,0:6]
        Predictions_table[(i+1)*100+j,28:34]=Predictions_microsoft[j,0:6]
        Predictions_table[(i+1)*100+j,35:41]=Predictions_microsoftlr2[j,0:6]   

#SVM
clf=svm.SVC(decision_function_shape='ovr')
clf.fit(Predictions_table,Labels_vertical)

Predictions=np.zeros((3500,1))
#for i in np.arange(3500):
Predictions=clf.predict(Predictions_table)

inc=0
for i in range(Labels_vertical.size):
    if (int(Labels_vertical[i]) == Predictions[i]):
        inc = inc +1
tr = (inc +0.0) / (Labels.size +0.0)

print tr
del Predictions

[[ 0.]
 [ 5.]
 [ 6.]
 ..., 
 [ 0.]
 [ 5.]
 [ 6.]]
0.999142857143


In [34]:
##train un SVM sur les predictions précédentes sur le public test
N_Batch = 35
taille_batch= 100
outfile_lenet=TP_root+"predictions/public_predictions_lenet"
outfile_monoconv=TP_root+"predictions/public_predictions_monoconv"
outfile_tripleconv=TP_root+"predictions/public_predictions_tripleconv"
outfile_3conv2ip=TP_root+"predictions/public_predictions_3conv2ip"
outfile_microsoft=TP_root+"predictions/public_predictions_microsoft"
outfile_microsoftlr2=TP_root+"predictions/public_predictions_microsoftlr2"

outfile_labels=TP_root+"public_labels"
Labels=pickle.load(open(outfile_labels,"r"))
Labels_vertical=np.zeros((3500,1))
for i in np.arange(3500):
    Labels_vertical[i,0]=Labels[0,i]
print Labels_vertical

Predictions_table=np.zeros((3500,6*7-1))

##Créer un tableau de prédictions [3500 * 41]. Ligne = entrées. Colonne = features (prédictions des NN mis bout à bout).
#Premier run initial
Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_0","r"))
Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_0","r"))
Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_0","r"))
Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_0","r"))
Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_0","r"))
Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_0","r"))
for i in np.arange(100):
    Predictions_table[i,0:6]=Predictions_lenet[i,0:6]
    Predictions_table[i,7:13]=Predictions_monoconv[i,0:6]
    Predictions_table[i,14:20]=Predictions_tripleconv[i,0:6]
    Predictions_table[i,21:27]=Predictions_3conv2ip[i,0:6]
    Predictions_table[i,28:34]=Predictions_microsoft[i,0:6]
    Predictions_table[i,35:41]=Predictions_microsoftlr2[i,0:6]

for i in np.arange(N_Batch-1):
    Predictions_lenet = pickle.load(open(outfile_lenet+"_batch_"+str(i+1),"r"))
    Predictions_monoconv = pickle.load(open(outfile_monoconv+"_batch_"+str(i+1),"r"))
    Predictions_tripleconv = pickle.load(open(outfile_tripleconv+"_batch_"+str(i+1),"r"))
    Predictions_3conv2ip= pickle.load(open(outfile_3conv2ip+"_batch_"+str(i+1),"r"))
    Predictions_microsoft= pickle.load(open(outfile_microsoft+"_batch_"+str(i+1),"r"))
    Predictions_microsoftlr2= pickle.load(open(outfile_microsoftlr2+"_batch_"+str(i+1),"r"))
    for j in np.arange(100):
        Predictions_table[(i+1)*100+j,0:6]=Predictions_lenet[j,0:6]
        Predictions_table[(i+1)*100+j,7:13]=Predictions_monoconv[j,0:6]
        Predictions_table[(i+1)*100+j,14:20]=Predictions_tripleconv[j,0:6]
        Predictions_table[(i+1)*100+j,21:27]=Predictions_3conv2ip[j,0:6]
        Predictions_table[(i+1)*100+j,28:34]=Predictions_microsoft[j,0:6]
        Predictions_table[(i+1)*100+j,35:41]=Predictions_microsoftlr2[j,0:6]   

Predictions=np.zeros((3500,1))
#for i in np.arange(3500):
Predictions=clf.predict(Predictions_table)

inc=0
for i in range(Labels_vertical.size):
    if (int(Labels_vertical[i]) == Predictions[i]):
        inc = inc +1
tr = (inc +0.0) / (Labels.size +0.0)

print tr
del Predictions

[[ 0.]
 [ 1.]
 [ 4.]
 ..., 
 [ 0.]
 [ 0.]
 [ 2.]]
0.170857142857
